# Acceso programático a Uniprot

Uniprot también ofrece una REST-API para acceder a sus datos.

Esta interfaz es bastante mas sencilla que la de NCBI.

La url de uniprot es "https://www.uniprot.org/uniprot", 
pero en algunos sistemas no puede usarse correctamente.
En su lugar podemos usar "https://histidine.dbi.udel.edu/uniprot/".

In [ ]:
library(httr)

In [ ]:
# base_url   <- "https://histidine.dbi.udel.edu/uniprot/"
base_url   <- "https://www.uniprot.org/uniprot/"

Al igual que NCBI, podemos hacer búsquedas por texto, especificando campos lo hacíamos en NCBI, aunque con un formato diferente.

Para escribir nuestra búsqueda tenemos que conocer los campos disponibles:

Estos están en: https://www.uniprot.org/help/query-fields.

El texto de búsqueda tiene que tener una cierta sintaxis: https://www.uniprot.org/help/text-search.

Además tenemos que buscar definir el formato en el que queremos los datos: Tipicamente, en formato tabular y fasta para las secuencias.

Por último que tenemos que definir que datos queremos recuperar (columnas): https://www.uniprot.org/help/uniprotkb_column_names.

Por ejemplo: queremos recuperar datos de proteínas polymerases de E. coli que pertenezcan a Swiss-Prot (es decir que estén curadas manualmente).

In [ ]:
query <- 'name:RNA polymerase AND organism:"Escherichia coli" AND reviewed:yes'

respuesta <- GET(
    base_url,
    query=list(query=query, format="tab", columns="id,entry name,protein names")
)


In [ ]:
ecoli_pol <- read.table(text=content(respuesta), header=TRUE, sep="\t")
head(ecoli_pol)

In [ ]:
# Podemos recuperar las secuencias de los registros de a uno
query <- paste(paste("accession:", ecoli_pol$Entry[1]), collapse=" ")
respuesta <- GET(
    base_url,
    query=list(query=query, format="fasta")
)


In [ ]:
cat(content(respuesta))

In [ ]:
# O podemos usar un servicio especial para recuperar varias secuencias.
retrieve_url <- 'https://www.uniprot.org/uploadlists'
respuesta <- GET(
    url=retrieve_url,
    query=list(
        query=paste(ecoli_pol$Entry,collapse=" "),
        format="fasta",
        from='ACC+ID',
        to='ACC'
    )
)

In [ ]:
cat(content(respuesta, as="text"))

In [ ]:
# install.packages('seqinr')
# Para leer formato fasta
library(seqinr)
sequences <- read.fasta(file=textConnection(content(respuesta, as="text")))

sequences[[1]]

# summary(sequences[[1]])

# attributes(sequences[[1]])

# attributes(sequences[[1]])$Name

## Servicio IDMapping

Uniprot tiene un servicio que permite mapear identificadores entre diferentes bases de datos.

Es necesario pasarle un lista de identificadores, y decir cuales son las bases de datos (o tipo de identificadores)
de origen y destino.

Lista de los tipos de identificadores que se pueden usar están en: https://www.uniprot.org/help/api_idmapping.

In [ ]:
retrieve_url <- 'https://www.uniprot.org/uploadlists'
respuesta <- GET(
    url=retrieve_url,
    query=list(
        query=paste(ecoli_pol$Entry,collapse=" "),
        format="tab",
        from='ACC+ID',
        to='PDB_ID'
    )
)


In [ ]:
mapping <- head(read.table(text=content(respuesta), header=TRUE))

In [ ]:
mapping

## Paquetes de R para conectarse a Uniprot

Hay dos paquetes de R que permiten conectarse a Uniprot:

- Uniprot.ws: https://www.bioconductor.org/packages/release/bioc/html/UniProt.ws.html
- UniprotR: https://cran.r-project.org/web/packages/UniprotR/index.html

No tienen una funcionalidad muy completa, por ejemplo carecen de la capacidad de hacer búsquedas de texto.

Aunque permiten descargar datos si se tienen los identificadores de los registros.